## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
from autogenstudio.teammanager import TeamManager

wm = TeamManager()
result = await wm.run(task="What is the weather in New York?", team_config="team.json")
print(result)

/Users/victordibia/projects/hax/autogen/python/packages/autogen-core/src/autogen_core/_component_config.py:252: UserWarning: 
⚠️  SECURITY WARNING ⚠️
Loading a FunctionTool from config will execute code to import the provided global imports and and function code.
Only load configs from TRUSTED sources to prevent arbitrary code execution.
  instance = component_class._from_config(validated_config)  # type: ignore


task_result=TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=47), metadata={}, content="I'm unable to provide real-time weather information. I recommend checking a reliable weather website or app for the current weather in New York. If you have any other questions or need assistance with a different topic, feel free to ask!", type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=138, completion_tokens=5), metadata={}, content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned") usage='' duration=1.734266996383667


In [2]:
result_stream =  wm.run_stream(task="What is the weather in New York?", team_config="team.json")
async for response in result_stream:
    print(response)

source='user' models_usage=None metadata={} content='What is the weather in New York?' type='TextMessage'
source='assistant_agent' models_usage=RequestUsage(prompt_tokens=86, completion_tokens=34) metadata={} content="I currently don't have access to real-time weather data. You can check a reliable weather website or application for the most current information about the weather in New York." type='TextMessage'
source='llm_call_event' models_usage=None metadata={} content='{"type": "LLMCall", "messages": [{"content": "You are a helpful assistant. Solve tasks carefully. When done, say TERMINATE.", "role": "system"}, {"content": "What is the weather in New York?", "role": "user", "name": "user"}], "response": {"id": "chatcmpl-B76eHFx8OqWQlbnNaK3W6CJtuznmR", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "I currently don\'t have access to real-time weather data. You can check a reliable weather website or application for the most current informa

## Load Directly Using the AgentChat API 

In [3]:
import json 
from autogen_agentchat.teams import BaseGroupChat
team_config = json.load(open("team.json"))  
team = BaseGroupChat.load_component(team_config)
print(team._participants)

## AutoGen Studio Database API

Api for creating objects and serializing to a database.

In [15]:
from autogenstudio.database import DatabaseManager
import os
# delete database
# if os.path.exists("test.db"):
#     os.remove("test.db")

os.makedirs("test", exist_ok=True)
# create a database
dbmanager = DatabaseManager(engine_uri="sqlite:///test.db", base_dir="test")
dbmanager.initialize_database()

2025-03-03 12:36:12.391 | INFO     | autogenstudio.database.db_manager:initialize_database:61 - Creating database tables...
2025-03-03 12:36:12.425 | INFO     | autogenstudio.database.schema_manager:_initialize_alembic:154 - Alembic initialization complete
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.autogenerate.compare] Detected type change from NUMERIC() to UUID() on 'message.run_id'
INFO  [alembic.autogenerate.compare] Detected type change from NUMERIC() to UUID() on 'run.id'


Response(message='Database initialized successfully', status=True, data=None)

## Sample AgentChat Example (Python)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

planner_agent = AssistantAgent(
    "planner_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request. Respond with a single sentence",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided. Respond with a single sentence",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.Respond with a single sentence",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.Respond with a single sentence",
)

termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)




In [7]:
result = group_chat.run_stream(task="Plan a 3 day trip to Nepal.")
async for response in result:
    print(response)

source='user' models_usage=None metadata={} content='Plan a 3 day trip to Nepal.' type='TextMessage'
source='planner_agent' models_usage=RequestUsage(prompt_tokens=45, completion_tokens=64) metadata={} content='Sure, on your 3-day trip to Nepal, you could start by visiting historical landmarks and local markets in Kathmandu, hike for a day in the Annapurna region to experience breathtaking mountain views and end your trip by exploring the religious sites and wildlife in Lumbini and Chitwan respectively.' type='TextMessage'
source='local_agent' models_usage=RequestUsage(prompt_tokens=125, completion_tokens=86) metadata={} content="Absolutely, on the first day, feel free to explore Kathmandu's rich history and culture, visit temples like Swayambhunath and Pashupatinath. On the second day, take a spectacular mountain flight to view Mount Everest and then head to Pokhara to engage in adventure activities or relax by Phewa Lake. On the third day, explore the biodiversity at Chitwan National

In [11]:
import json

# convert to config 
config = group_chat.dump_component().model_dump()
# save as json 

with open("travel_team.json", "w") as f:
    json.dump(config, f, indent=4)

# load from json
with open("travel_team.json", "r") as f:
    config = json.load(f)

group_chat = RoundRobinGroupChat.load_component(config) 
result = group_chat.run_stream(task="Plan a 3 day trip to Nepal.") 
async for response in result:
    print(response)

source='user' models_usage=None metadata={} content='Plan a 3 day trip to Nepal.' type='TextMessage'
source='planner_agent' models_usage=RequestUsage(prompt_tokens=45, completion_tokens=74) metadata={} content='For a 3-day trip to Nepal, start your journey in Kathmandu, visit renowned UNESCO World Heritage Sites including Pashupatinath Temple and Boudhanath Stupa, day two explore the breathtaking city of Pokhara, and on the final day, hike the serene trails of Nagarkot to witness a captivating sunrise over the Himalayas.' type='TextMessage'
source='local_agent' models_usage=RequestUsage(prompt_tokens=135, completion_tokens=57) metadata={} content='Immerse yourself in the local lifestyle by visiting the bustling markets of Kathmandu, explore the ancient, history-rich city of Bhaktapur, take a day hike to the breathtaking monasteries in the mountains and wrap it up with an unforgettable traditional Nepalese dinner.' type='TextMessage'
source='language_agent' models_usage=RequestUsage(pro